In [1]:
import numpy as np
import pandas as pd
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=64
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=64
EVALUATE_EVERY=15
FILTER_SIZES=[3,4,5]
L2_REG_LAMBDA=0.001
LOG_DEVICE_PLACEMENT=False
NUM_EPOCHS=20
NUM_FILTERS=128
fname = "w2vmodel" #w2vmodel128

In [2]:
def washstr(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [3]:
import nltk
import string
import re

def tokenizer(text):
    lower_txt = washstr(text)
    return lower_txt.split(" ")

In [4]:
def extfeat(X_train):
    neg = pd.read_csv('negative-words.txt')['neg'].values
    pos = pd.read_csv('positive-words.txt')['pos'].values
    neg = set(neg)
    pos = set(pos)
    
    ext = []
    for i in range(len(X_train)):
        p = 0.0
        n = 0.0
        total = 0.0
        for j in X_train[i]:
            if j != " ":
                total += 1
                if j in pos:
                    p += 1
                if j in neg:
                    n += 1
        if n == 0:
            ratio = p
        else:
            ratio = p/n
        ext.append([p,n,p/total,n/total,ratio])
    return ext

In [5]:
import gensim
def word2vec(X_train):
    X = []
    for i in range(X_train.shape[0]):
        X.append(tokenizer(X_train[i]))
    
    for i in range(len(X)):
        while len(X[i]) < max_len:
            X[i].append(" ")
        
    model = gensim.models.Word2Vec(X, min_count=1,size=EMBEDDING_DIM)
    return model

In [6]:
def getwvec(X_train,model):
    x = []
    for i in range(len(X_train)):
        x.append(model.wv[X_train[i]])
    return np.array(x)

In [7]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    data = np.array(data)
    data_size = len(data)
    print(data_size)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [8]:
def train_step(x_batch, y_batch):
    feed_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: DROPOUT_KEEP_PROB
    }
    _, step, summaries, loss, accuracy = sess.run(
        [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)
    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

def dev_step(x_batch, y_batch):
    feed_dict = {
      cnn.input_x: x_batch,
      cnn.input_y: y_batch,
      cnn.dropout_keep_prob: 1.0
    }
    step, summaries, loss, accuracy = sess.run(
        [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)
    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

In [9]:
train_set = pd.read_csv('labeledmusic.csv')

In [10]:
lytrain = train_set['text'].values 

y_train = train_set['mood'].values

In [11]:
print(lytrain.shape, y_train.shape)

(1199,) (1199,)


In [12]:
X_train = []
Y = []

max_len = 0

for i in range(lytrain.shape[0]):
    X_train.append(tokenizer(lytrain[i]))
    max_len = max(max_len, len(X_train[i]))

    if y_train[i] == "sad":
        Y.append([0,1])
    else:
        Y.append([1,0])

In [13]:
print(len(X_train), max_len)

1199 1266


In [14]:
# fill each lyric, to make it of 1087 tokens
for i in range(len(X_train)):
    while len(X_train[i]) < max_len:
        X_train[i].append(" ")

In [15]:
for i in range(len(X_train)):
    if len(X_train[i]) != max_len:
        print("error")

In [16]:
print(lytrain.shape)

(1199,)


In [17]:
# # since the dataset is too small, word vector may not be good, so use a greater lyric set to generate w2v model
# unlabeled = pd.read_csv('songdata.csv')
# w2vtran = unlabeled['text'].values
# w2vtran = np.row_stack((w2vtran.reshape(-1,1),lytrain.reshape(-1,1)))

In [18]:
# model = word2vec(w2vtran[:,0])
# model.save(fname)

In [19]:
model = gensim.models.Word2Vec.load(fname)

In [20]:
# print(w2vtran.shape)

In [21]:
x_ = getwvec(X_train,model)
print(x_.shape)

(1199, 1266, 64)


In [22]:
# extra = extfeat(X_train)

In [23]:
# extra = np.array(extra)
# print(extra.shape)

(1199, 5)


In [25]:
# x_ = x_.reshape(-1,x_.shape[1]*x_.shape[2])

In [27]:
# x = np.column_stack((x_,extra))

In [28]:
# from sklearn.decomposition import PCA 
# pca=PCA(n_components=0.998)
# x = pca.fit_transform(x)

In [22]:
x = x_
y = np.array(Y)

In [23]:
print(x.shape, y.shape)

(1199, 1266, 64) (1199, 2)


In [24]:
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
dev_sample_index = -1 * int(DEV_SAMPLE_PERCENTAGE * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

del x, y, x_shuffled, y_shuffled

print("yTrain/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
print("xTrain/Dev split: {:d}/{:d}".format(len(x_train), len(x_dev)))

yTrain/Dev split: 1080/119
xTrain/Dev split: 1080/119


In [ ]:
#! /usr/bin/env python

import tensorflow as tf
import numpy as np
import time
import datetime
from text_cnn import TextCNN

# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=ALLOW_SOFT_PLACEMENT,
      log_device_placement=LOG_DEVICE_PLACEMENT)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            embedding_dim=EMBEDDING_DIM,
            filter_sizes=FILTER_SIZES,
            num_filters=NUM_FILTERS,
            l2_reg_lambda=L2_REG_LAMBDA)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])

        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        
        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), BATCH_SIZE, NUM_EPOCHS)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            print(len(x_batch))
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % EVALUATE_EVERY == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev)
                print("")


INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using conv-maxpool-4/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/sparsity is illegal; using conv-maxpool-4/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-5/W:0/grad/his